<h1 style="font-size:200%; font-family:cursive; color:white;">1. Import Required Libraries & Dataset</h1>

In [1]:
import numpy as np
import pandas as pd
import ast
import time
import datetime
import torch
import torch.nn as nn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from transformers import AutoModel, BertTokenizer

#specify GPU
device = torch.device("cuda")

### Check if Pytorch is using GPU

In [2]:
torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.current_device(), torch.cuda.device(0), torch.cuda.get_device_name(0)

(True,
 1,
 0,
 'NVIDIA RTX 3500 Ada Generation Laptop GPU')

In [4]:
df_videos = pd.read_csv("/app/AI-Module/Resources/Datasets/how2sign.csv")

In [5]:
df_videos['points'] = df_videos['points'].apply(ast.literal_eval)

In [ ]:
import sys
sys.path.append("/app/AI-Module/Modules")
from VideoFormaterClass import VideoFormater

In [ ]:
vf = VideoFormater()
vf.concatAndExportVideos(df_videos, "/app/AI-Module/Resources/Datasets/how2sign2.csv")

In [ ]:
max_len = df_videos['points'].apply(lambda x: len(x)).max()
df_videos.head(), max_len

In [13]:
def add_padding(max_frames, pointSeries: pd.Series):
    for i in range(len(pointSeries)):
        current_length = len(pointSeries[i])
        if current_length < max_frames:
            padding = np.full(
                (max_frames - current_length, 2172), 
                -1
            )
            padding[:, 3::4] = 0
            pointSeries[i] = np.concatenate((pointSeries[i], padding), axis=0)

    return pointSeries

In [ ]:
df_videos['points'] = add_padding(max_len, df_videos['points'])

In [ ]:
X_df = df_videos['points'].to_frame()
y_df = df_videos['translation'].to_frame()
seed = 31991
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=(100/500), random_state=seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=(100/400), random_state=seed)

In [ ]:
X_train.head(), y_train.head(), X_val.head(), y_val.head(), X_test.head(), y_test.head()

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

<h1 style="font-size:200%; font-family:cursive; color:white;">3. Import Bert - base- uncased</h1>

In [ ]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [16]:
# tokenize and encode sequences in the training set
tokens_label_train = tokenizer.batch_encode_plus(
    y_train['translation'].tolist(),
    padding = True
)

# tokenize and encode sequences in the validation set
tokens_label_val = tokenizer.batch_encode_plus(
    y_val['translation'].tolist(),
    padding = True
)

# tokenize and encode sequences in the test set
tokens_label_test = tokenizer.batch_encode_plus(
    y_test['translation'].tolist(),
    padding = True
)

<u><h2 style="font-size:170%; font-family:cursive;">What is the maximum sequence length of the input?</h2></u>

<p style="font-size:150%; font-family:verdana;">The maximum sequence length of the input = 512</p>

<h1 style="font-size:200%; font-family:cursive; color:white;">5. List to Tensors</h1>

In [8]:
def create_attention_mask_from_points(seq_tensor):
    mask = torch.ones(seq_tensor.shape, dtype=torch.long)
    missing_data = (seq_tensor[..., :3] == -1).all(dim=-1) & (seq_tensor[..., 3] == 0)
    mask[missing_data] = 0
    return mask

In [9]:
# convert lists to tensors

train_seq = torch.stack([torch.tensor(seq) for seq in X_train['points'].tolist()])
train_mask = create_attention_mask_from_points(train_seq)
train_y = tokens_label_train['input_ids']

val_seq = torch.stack([torch.tensor(seq) for seq in X_val['points'].tolist()])
val_mask = create_attention_mask_from_points(val_seq)
val_y = tokens_label_val['input_ids']

test_seq = torch.stack([torch.tensor(seq) for seq in X_test['points'].tolist()])
test_mask = create_attention_mask_from_points(test_seq)
test_y = tokens_label_test['input_ids']

<h1 style="font-size:200%; font-family:cursive; color:white;">6. Data Loader</h1>

In [ ]:

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

<h1 style="font-size:200%; font-family:cursive; color:white;">7. Model Architecture</h1>

In [ ]:
vocab_size = tokenizer.vocab_size

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert, vocab_size):
        super(BERT_Arch, self).__init__()
        
        self.bert = bert 
        
        # Dropout layer
        self.dropout = nn.Dropout(0.1)
        
        # ReLU activation function
        self.relu = nn.ReLU()

        # Dense layer 1
        self.fc1 = nn.Linear(768, 512)
        
        # Dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512, vocab_size)  # Use tokenizer's vocab size

        # Softmax activation function
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, seq_input, mask):
        outputs = self.bert(seq_input, attention_mask=mask, return_dict=False)
        
        # Use the last hidden state for each token (outputs[0])
        x = self.fc1(outputs[0])  # (batch_size, sequence_length, 512)

        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)  # (batch_size, sequence_length, vocab_size)
        
        # Apply softmax activation (for each token in the sequence)
        x = self.softmax(x)

        return x


In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert, vocab_size)

# push the model to GPU
model = model.to(device)

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(),lr = 1e-5) 

In [ ]:
# Define the loss function
cross_entropy = nn.CrossEntropyLoss()

# number of training epochs
epochs = 10

<h1 style="font-size:200%; font-family:cursive; color:white;">8. Fine - Tune</h1>

In [ ]:
%pip install nltk

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Define the BLEU score function for a single prediction and reference
def calculate_bleu(preds, references):
    smoothie = SmoothingFunction().method4  # Smoothing method to avoid zero scores for short sentences
    score = sentence_bleu([references], preds, smoothing_function=smoothie)
    return score

# Function to convert token ids back to words (predictions and labels)
def decode_predictions(predictions, tokenizer):
    # Convert token IDs to tokens (words) using the tokenizer's decode method
    return [tokenizer.decode(pred, skip_special_tokens=True) for pred in predictions]

def train():
    
    model.train()
    total_loss = 0
    total_preds = []
    total_bleu_score = 0  # To accumulate BLEU scores
    
    # iterate over batches
    for step, batch in enumerate(train_dataloader):
        
        # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print(f'  Batch {step:>5,} of {len(train_dataloader):>5,}.')

        # push the batch to GPU
        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch
        
        # clear previously calculated gradients 
        model.zero_grad()        

        # get model predictions for the current batch
        preds = model(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

        # add on to the total loss
        total_loss += loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the gradients to 1.0 to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # move model predictions to CPU for further processing
        preds = preds.detach().cpu().numpy()
        
        # append predictions for final concatenation (reshaping later)
        total_preds.append(preds)

        # Decode predictions and labels to human-readable sequences
        decoded_preds = decode_predictions(preds.argmax(axis=-1), tokenizer)
        decoded_labels = decode_predictions(labels.cpu().numpy(), tokenizer)

        # Calculate BLEU score for this batch (by averaging the scores for each sentence)
        batch_bleu_score = sum([calculate_bleu(pred, ref) for pred, ref in zip(decoded_preds, decoded_labels)]) / len(decoded_preds)
        total_bleu_score += batch_bleu_score

    # Average loss for this epoch
    avg_loss = total_loss / len(train_dataloader)

    # Concatenate all predictions
    total_preds = np.concatenate(total_preds, axis=0)

    # Average BLEU score for this epoch
    avg_bleu_score = total_bleu_score / len(train_dataloader)

    # return the average loss and average BLEU score
    return avg_loss, avg_bleu_score


In [ ]:
def evaluate():
    print("\nEvaluating...")

    # Set the model to evaluation mode (deactivates dropout)
    model.eval()

    total_loss, total_bleu_score = 0, 0
    total_preds = []

    # Start time for progress update
    t0 = time.time()

    # Iterate over batches
    for step, batch in enumerate(val_dataloader):
        
        # Progress update every 50 batches
        if step % 50 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f'  Batch {step:>5,}  of  {len(val_dataloader):>5,}.    Elapsed: {elapsed}.')
        
        # Push the batch to GPU
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch

        # Deactivate autograd for evaluation
        with torch.no_grad():
            # Model predictions
            preds = model(sent_id, mask)

            # Compute the validation loss
            loss = cross_entropy(preds, labels)
            total_loss += loss.item()

            # Move predictions to CPU
            preds = preds.detach().cpu().numpy()

            # Store predictions
            total_preds.append(preds)

            # Decode predictions and labels
            decoded_preds = decode_predictions(preds.argmax(axis=-1), tokenizer)
            decoded_labels = decode_predictions(labels.cpu().numpy(), tokenizer)

            # Calculate BLEU score for the batch
            batch_bleu_score = sum([calculate_bleu(pred, ref) for pred, ref in zip(decoded_preds, decoded_labels)]) / len(decoded_preds)
            total_bleu_score += batch_bleu_score

    # Compute the average validation loss
    avg_loss = total_loss / len(val_dataloader)

    # Compute the average BLEU score
    avg_bleu_score = total_bleu_score / len(val_dataloader)

    # Concatenate all predictions
    total_preds = np.concatenate(total_preds, axis=0)

    # Return the average loss and BLEU score
    return avg_loss, avg_bleu_score


In [ ]:
# set initial loss and BLEU score to infinite/zero
best_valid_loss = float('inf')
best_bleu_score = 0

# empty lists to store training and validation loss and BLEU score of each epoch
train_losses = []
valid_losses = []
train_bleu_scores = []
valid_bleu_scores = []

# for each epoch
for epoch in range(epochs):
    
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    # train model
    train_loss, train_bleu = train()
    
    # evaluate model
    valid_loss, valid_bleu = evaluate()
    
    # save the best model based on validation loss
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    # Optionally: Save the model based on BLEU score
    # if valid_bleu > best_bleu_score:
    #     best_bleu_score = valid_bleu
    #     torch.save(model.state_dict(), 'saved_weights_best_bleu.pt')
    
    # append training and validation loss and BLEU score
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    train_bleu_scores.append(train_bleu)
    valid_bleu_scores.append(valid_bleu)
    
    # print training and validation statistics
    print(f'\nTraining Loss: {train_loss:.3f} | Training BLEU: {train_bleu:.3f}')
    print(f'Validation Loss: {valid_loss:.3f} | Validation BLEU: {valid_bleu:.3f}')

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<h1 style="font-size:200%; font-family:cursive; color:white;">9. Make Predictions</h1>

In [ ]:
# get predictions for test data
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()

In [ ]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))